In [2]:
from sklearn.utils import shuffle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from PIL import Image, ImageEnhance
from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.losses import *
from tensorflow.keras.models import *
from tensorflow.keras.metrics import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.applications import *
from tensorflow.keras.preprocessing.image import load_img
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
# from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
import os
print(tf.config.list_physical_devices('GPU'))
import random

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2024-01-14 12:10:58.625218: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-14 12:10:58.658626: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-14 12:10:58.658863: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [3]:
def augment_image(image):
    image = Image.fromarray(np.uint8(image))
    image = ImageEnhance.Brightness(image).enhance(random.uniform(0.8,1.2))
    image = ImageEnhance.Contrast(image).enhance(random.uniform(0.8,1.2))
    image = ImageEnhance.Sharpness(image).enhance(random.uniform(0.8,1.2))
    image = np.array(image)/255.0
    return image

In [4]:
train_data_path = 'Training/'
filepaths =[]
labels = []
import pandas as pd
import os
folds = os.listdir(train_data_path)

for fold in folds:
    f_path = os.path.join(train_data_path , fold)
    filelists = os.listdir(f_path)
    
    for file in filelists:
        filepaths.append(os.path.join(f_path , file))
        labels.append(fold)
        
#Concat data paths with labels
Fseries = pd.Series(filepaths , name = 'filepaths')
Lseries = pd.Series(labels , name = 'label')
train_df = pd.concat([Fseries , Lseries] , axis = 1)
test_data_path = 'Testing/'
filepaths =[]
labels = []

folds = os.listdir(test_data_path)

for fold in folds:
    f_path = os.path.join(test_data_path , fold)
    filelists = os.listdir(f_path)
    
    for file in filelists:
        filepaths.append(os.path.join(f_path , file))
        labels.append(fold)
        
Fseries = pd.Series(filepaths , name = 'filepaths')
Lseries = pd.Series(labels , name = 'label')
test_df = pd.concat([Fseries , Lseries] , axis = 1)

In [5]:
valid , test = train_test_split(test_df , train_size = 0.8, shuffle = True , random_state= 42)

In [6]:
img_size = (224, 244)
batch_size = 32

tr_gen = ImageDataGenerator(
)
ts_gen = ImageDataGenerator(
)

train_gen = tr_gen.flow_from_dataframe(
    train_df,
    x_col="filepaths",
    y_col="label",
    target_size=img_size,
    class_mode="categorical",
    color_mode="rgb",
    shuffle=True,
    batch_size=batch_size,
)
valid_gen = ts_gen.flow_from_dataframe(
    valid,
    x_col="filepaths",
    y_col="label",
    target_size=img_size,
    class_mode="categorical",
    color_mode="rgb",
    shuffle=True,
    batch_size=batch_size,
)
test_gen = ts_gen.flow_from_dataframe(
    test,
    x_col="filepaths",
    y_col="label",
    target_size=img_size,
    class_mode="categorical",
    color_mode="rgb",
    shuffle=False,
    batch_size=batch_size,
)

Found 5712 validated image filenames belonging to 4 classes.
Found 1048 validated image filenames belonging to 4 classes.
Found 263 validated image filenames belonging to 4 classes.


In [7]:
img_shape = (img_size[0], img_size[1], 3)
model = Sequential(
    [
        tf.keras.layers.BatchNormalization(),
        Conv2D(  
            filters=96,
            kernel_size=(11, 11),
            strides=(4, 4),
            activation="relu",
            input_shape=img_shape,
        ),
        MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),
        Conv2D(
            filters=256,
            kernel_size=(5, 5),
            strides=(1, 1),
            activation="relu",
            padding="same",
        ),
        MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),
        Conv2D(
            filters=384,
            kernel_size=(3, 3),
            strides=(1, 1),
            activation="relu",
            padding="same",
        ),
        Conv2D(
            filters=384,
            kernel_size=(3, 3),
            strides=(1, 1),
            activation="relu",
            padding="same",
        ),
        MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),
        Conv2D(
            filters=384,
            kernel_size=(3, 3),
            strides=(1, 1),
            activation="relu",
            padding="same",
        ),
        Conv2D(
            filters=256,
            kernel_size=(3, 3),
            strides=(1, 1),
            activation="relu",
            padding="same",
        ),
        MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),
        Flatten(),
        Dense(4096, activation="relu"),
        Dense(4096, activation="relu"),
        Dense(4, activation="softmax"),
    ]
)

model.compile(
    loss="categorical_crossentropy",
    optimizer=Adam(learning_rate=0.0001),
    metrics=["accuracy"],
)

history = model.fit(
    train_gen,
    steps_per_epoch=train_gen.samples / train_gen.batch_size,
    validation_data=valid_gen,
    epochs=20, 
    verbose=1,
)

2024-01-14 12:10:58.755683: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-14 12:10:58.756045: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-14 12:10:58.756292: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Epoch 1/20


2024-01-14 12:11:03.833180: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f8a68f5e700 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-14 12:11:03.833215: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 2050, Compute Capability 8.6
2024-01-14 12:11:03.837173: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1705214463.908244   37510 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2024-01-14 12:11:07.981204: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.05GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
20

178/178 [==============================] - 33s 127ms/step - loss: 0.8254 - accuracy: 0.6628 - val_loss: 0.6681 - val_accuracy: 0.7223
Epoch 2/20
178/178 [==============================] - 16s 90ms/step - loss: 0.4020 - accuracy: 0.8442 - val_loss: 0.4096 - val_accuracy: 0.8225
Epoch 3/20
178/178 [==============================] - 15s 84ms/step - loss: 0.2968 - accuracy: 0.8899 - val_loss: 0.2549 - val_accuracy: 0.8989
Epoch 4/20
178/178 [==============================] - 15s 84ms/step - loss: 0.1888 - accuracy: 0.9333 - val_loss: 0.1802 - val_accuracy: 0.9399
Epoch 5/20
178/178 [==============================] - 15s 84ms/step - loss: 0.1359 - accuracy: 0.9536 - val_loss: 0.2307 - val_accuracy: 0.9160
Epoch 6/20
178/178 [==============================] - 15s 84ms/step - loss: 0.1204 - accuracy: 0.9569 - val_loss: 0.1077 - val_accuracy: 0.9618
Epoch 7/20
178/178 [==============================] - 15s 84ms/step - loss: 0.0650 - accuracy: 0.9795 - val_loss: 0.2046 - val_accuracy: 0.9237
Ep

In [8]:
train_score = model.evaluate(train_gen , steps =16 , verbose = 1)
valid_score = model.evaluate(valid_gen , steps = 16 , verbose = 1)
test_score = model.evaluate(test_gen , steps = 16 , verbose = 1)

print("Train Loss: ", train_score[0])
print("Train Accuracy: ", train_score[1])
print('-' * 20)
print("Validation Loss: ", valid_score[0])
print("Validation Accuracy: ", valid_score[1])
print('-' * 20)
print("Test Loss: ", test_score[0])
print("Test Accuracy: ", test_score[1])

16/16 [==============================] - 1s 39ms/step - loss: 0.2118 - accuracy: 0.9430
Train Loss:  0.025473223999142647
Train Accuracy:  0.986328125
--------------------
Validation Loss:  0.11318159848451614
Validation Accuracy:  0.96484375
--------------------
Test Loss:  0.21183985471725464
Test Accuracy:  0.9429658055305481


2024-01-14 12:16:45.524368: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 16758032394245800167
2024-01-14 12:16:45.524414: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 6343895248368649371
2024-01-14 12:16:45.524436: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 9221365843799908292


In [9]:
model.save("ninetyheightdmodel")

2024-01-14 12:16:46.313244: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 67108864 exceeds 10% of free system memory.
2024-01-14 12:16:46.396187: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 67108864 exceeds 10% of free system memory.
2024-01-14 12:16:46.426950: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 67108864 exceeds 10% of free system memory.


INFO:tensorflow:Assets written to: ninetyheightdmodel/assets


INFO:tensorflow:Assets written to: ninetyheightdmodel/assets


In [10]:
# plt.figure(figsize=(8,4))
# plt.grid(True)
# plt.plot(history.history['categorical_accuracy'], '.g-', linewidth=2)
# plt.plot(history.history['loss'], '.r-', linewidth=2)
# plt.title('Model Training History')
# plt.xlabel('epoch')
# plt.xticks([x for x in range(10)])
# plt.legend(['Accuracy', 'Loss'], loc='upper left', bbox_to_anchor=(1, 1))
# plt.show()

In [11]:
preds = model.predict(test_gen)
y_pred = np.argmax(preds , axis = 1)
g_dict = test_gen.class_indices
classes = list(g_dict.keys())
print(classification_report(test_gen.classes,y_pred))

9/9 [==============================] - 0s 32ms/step
              precision    recall  f1-score   support

           0       1.00      0.92      0.96        61
           1       0.90      0.87      0.89        63
           2       0.92      1.00      0.96        79
           3       0.97      0.97      0.97        60

    accuracy                           0.94       263
   macro avg       0.95      0.94      0.94       263
weighted avg       0.94      0.94      0.94       263


In [12]:
tf.keras.utils.plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.
